# Opioids Project

Ra'Kira Nelson and Alexa Fahrer

In [1]:
import pandas as pd

pd.set_option("mode.copy_on_write", True)

In [2]:
prescriptions = pd.read_parquet("ids590_opioids_by_drug_county_year.parquet")

In [3]:
deaths_2003 = pd.read_csv(
    "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%202003.txt",
    sep="\t",
    skipfooter=15,
    engine="python",
)
deaths_2004 = pd.read_csv(
    "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%202004.txt",
    sep="\t",
    skipfooter=15,
    engine="python",
)
deaths_2005 = pd.read_csv(
    "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%202005.txt",
    sep="\t",
    skipfooter=15,
    engine="python",
)
deaths_2006 = pd.read_csv(
    "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%202006.txt",
    sep="\t",
    skipfooter=15,
    engine="python",
)
deaths_2007 = pd.read_csv(
    "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%202007.txt",
    sep="\t",
    skipfooter=15,
    engine="python",
)
deaths_2008 = pd.read_csv(
    "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%202008.txt",
    sep="\t",
    skipfooter=15,
    engine="python",
)
deaths_2009 = pd.read_csv(
    "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%202009.txt",
    sep="\t",
    skipfooter=15,
    engine="python",
)
deaths_2010 = pd.read_csv(
    "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%202010.txt",
    sep="\t",
    skipfooter=15,
    engine="python",
)
deaths_2011 = pd.read_csv(
    "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%202011.txt",
    sep="\t",
    skipfooter=15,
    engine="python",
)
deaths_2012 = pd.read_csv(
    "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%202012.txt",
    sep="\t",
    skipfooter=15,
    engine="python",
)
deaths_2013 = pd.read_csv(
    "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%202013.txt",
    sep="\t",
    skipfooter=15,
    engine="python",
)
deaths_2014 = pd.read_csv(
    "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%202014.txt",
    sep="\t",
    skipfooter=15,
    engine="python",
)
deaths_2015 = pd.read_csv(
    "https://media.githubusercontent.com/media/nickeubank/ids540_opioid_data/refs/heads/main/vitalstatistics/Underlying%20Cause%20of%20Death%2C%202015.txt",
    sep="\t",
    skipfooter=15,
    engine="python",
)

deaths_2003 = deaths_2003.drop(columns=["Notes"])
deaths_2004 = deaths_2004.drop(columns=["Notes"])
deaths_2005 = deaths_2005.drop(columns=["Notes"])
deaths_2006 = deaths_2006.drop(columns=["Notes"])
deaths_2007 = deaths_2007.drop(columns=["Notes"])
deaths_2008 = deaths_2008.drop(columns=["Notes"])
deaths_2009 = deaths_2009.drop(columns=["Notes"])
deaths_2010 = deaths_2010.drop(columns=["Notes"])
deaths_2011 = deaths_2011.drop(columns=["Notes"])
deaths_2012 = deaths_2012.drop(columns=["Notes"])
deaths_2013 = deaths_2013.drop(columns=["Notes"])
deaths_2014 = deaths_2014.drop(columns=["Notes"])
deaths_2015 = deaths_2015.drop(columns=["Notes"])

In [4]:
prescriptions

,buyer_state,buyer_county,year,drug_name,mme_conversion_factor,calc_base_wt_in_gm
0,AK,ANCHORAGE,2006,BUPRENORPHINE,30.0,125.154336
1,AK,ANCHORAGE,2006,BUPRENORPHINE,75.0,0.216367
2,AK,ANCHORAGE,2006,CODEINE,0.15,13362.99003
3,AK,ANCHORAGE,2006,DIHYDROCODEINE,0.25,4.27264
4,AK,ANCHORAGE,2006,FENTANYL,100.0,476.051283
...,...,...,...,...,...,...
563626,WY,WESTON,2019,METHADONE,4.0,25.4904
563627,WY,WESTON,2019,MORPHINE,1.0,332.57952
563628,WY,WESTON,2019,OXYCODONE,1.5,621.053064
563629,WY,WESTON,2019,OXYMORPHONE,3.0,9.63468


In [5]:
prescriptions["buyer_county"] = prescriptions["buyer_county"].str.capitalize()
prescriptions["drug_name"] = prescriptions["drug_name"].str.capitalize()

In [6]:
prescriptions_fl = prescriptions.loc[prescriptions["buyer_state"] == "FL"]
prescriptions_fl["buyer_county"].nunique()

67

In [7]:
deaths_2003

,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths
0,"Autauga County, AL",1001,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,397.0
1,"Baldwin County, AL",1003,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0
2,"Baldwin County, AL",1003,2003.0,2003.0,All other alcohol-induced causes,A9,14.0
3,"Baldwin County, AL",1003,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,1479.0
4,"Barbour County, AL",1005,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,287.0
...,...,...,...,...,...,...,...
4082,"Sweetwater County, WY",56037,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,272.0
4083,"Teton County, WY",56039,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,85.0
4084,"Uinta County, WY",56041,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,111.0
4085,"Washakie County, WY",56043,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,88.0


In [8]:
deaths_2003["Year"] = pd.to_numeric(deaths_2003["Year"], errors="coerce").astype(
    "Int64"
)
deaths_2003["Year Code"] = pd.to_numeric(
    deaths_2003["Year Code"], errors="coerce"
).astype("Int64")

In [ ]:
deaths_2003[["County Name", "State"]] = deaths_2003["County"].str.split(
    ",", n=1, expand=True
)
deaths_2003["County Name"] = deaths_2003["County Name"].str.strip()
deaths_2003["State"] = deaths_2003["State"].str.strip()
deaths_2003["County Name"] = (
    deaths_2003["County Name"].str.replace(r"\bCounty\b", "", regex=True).str.strip()
)

In [10]:
deaths_2003_fl = deaths_2003.loc[deaths_2003["State"] == "FL"]

In [11]:
prescriptions

,buyer_state,buyer_county,year,drug_name,mme_conversion_factor,calc_base_wt_in_gm
0,AK,Anchorage,2006,Buprenorphine,30.0,125.154336
1,AK,Anchorage,2006,Buprenorphine,75.0,0.216367
2,AK,Anchorage,2006,Codeine,0.15,13362.99003
3,AK,Anchorage,2006,Dihydrocodeine,0.25,4.27264
4,AK,Anchorage,2006,Fentanyl,100.0,476.051283
...,...,...,...,...,...,...
563626,WY,Weston,2019,Methadone,4.0,25.4904
563627,WY,Weston,2019,Morphine,1.0,332.57952
563628,WY,Weston,2019,Oxycodone,1.5,621.053064
563629,WY,Weston,2019,Oxymorphone,3.0,9.63468


In [12]:
deaths_2003

,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,County Name,State
0,"Autauga County, AL",1001,2003,2003,All other non-drug and non-alcohol causes,O9,397.0,Autauga,AL
1,"Baldwin County, AL",1003,2003,2003,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0,Baldwin,AL
2,"Baldwin County, AL",1003,2003,2003,All other alcohol-induced causes,A9,14.0,Baldwin,AL
3,"Baldwin County, AL",1003,2003,2003,All other non-drug and non-alcohol causes,O9,1479.0,Baldwin,AL
4,"Barbour County, AL",1005,2003,2003,All other non-drug and non-alcohol causes,O9,287.0,Barbour,AL
...,...,...,...,...,...,...,...,...,...
4082,"Sweetwater County, WY",56037,2003,2003,All other non-drug and non-alcohol causes,O9,272.0,Sweetwater,WY
4083,"Teton County, WY",56039,2003,2003,All other non-drug and non-alcohol causes,O9,85.0,Teton,WY
4084,"Uinta County, WY",56041,2003,2003,All other non-drug and non-alcohol causes,O9,111.0,Uinta,WY
4085,"Washakie County, WY",56043,2003,2003,All other non-drug and non-alcohol causes,O9,88.0,Washakie,WY
